# Assoziationsregeln

## Überblick

Assoziationen sind Beziehungen zwischen Objekten. Die Idee von Assoziationsregeln es also diese Beziehungen durch Regel zu beschreiben, mit dem Ziel zu Wissen welche weiteren Objekte zu den bisherigen Objekten passen. Bei den Assoziationsregeln spricht man statt von Objekten in der Regeln von *Gegenständen* (engl. *items*). Das Standardbeispiel für Assoziationsregeln ist die Analyse von Warenkörben und Einkäufen. Ein Kunde fügt Produkte (Gegenstände) dem Warenkorb hinzu. Sobald der Warenkorb nicht mehr leer ist, kann man den Inhalt nutzen um Assoziationen zu weiteren Produkten, die für den Kunden von Interesse herzustellen. Diese können dem Kunden dann zum Beispiel als Werbung vorgeschlagen werden. 

```{figure} images/associationsrules_general_german.png
---
width: 600px
name: fig-asm-example
---
Warenkörbe als Beispiel für Assoziationen.
```

In diesem Beispiel sind die Assoziationen definiert als "Produkte, die von Kunden gleichzeitig gekauft werden". Wenn man dies generalisiert hat man *Transaktionen*, wobei jede Transaktion eine Beobachtung von gemeinsam verarbeiteten Gegenständen ist. Mit Hilfe von *Assoziationsanalyse* werden Assoziationsregeln aus den Transaktionen gelernt.

```{figure} images/associationsrules_abstract_german.png
---
width: 600px
name: fig-asm-concept
---
Konzept von Assoziationsanalyse.
```

Die Assoziationsregeln sollte "interesante" Beziehungen beschreiben. Die Definition von interesant hängt vom Anwendungsfall ab. Im obigen Beispiel sind Beziehungen interesant, die Beschreiben was Kunden bewusst zusammen kaufen. Wir können die Assoziationsanalyse auch formal definieren. Gegeben ist eine Menge von Gegenständen, auch *Itemsets* gennant, definiert als $I = \{i_1, ..., i_m\}$ und  sowie eine Menge von Transaktionen $T = \{t_1, ..., t_n\}$, wobei Transaktion eine Teilmenge von Gegenständen beinhaltet, also $t_j \subseteq I, j=1, ..., n$. Assoziationsregeln sind logische Ausdrücke der Form $X \Rightarrow Y$, wobei $X$ und $Y$ disjunkte Teilmengen von Gegenständen sind, d.h., es gilt $X, Y \subseteq I$ und $X \cap Y = \emptyset$. Man nennt $X$ auch *Bedingung* (engl. *antecedent*) oder linke Seite der Regel und $Y$ auch *Folgerung* (engl. *consequent*) oder rechte Seite der Regel. 

> **Bemerkung:**
>
> Ihnen ist möglicherweise aufgefallen, dass wir bisher keine Merkmale benannt haben, sondern stattdessen von Gegenständen gesprochen haben. Außerdem gibt es auch keine Instanzen, sondern Transaktionen. Hierfür gibt es zwei Gründe: Zum einen ist das die übliche Terminologie der Assoziationsanalyse. Zum zweiten gibt es keine echten Merkmale, da jedes Objekt von seiner Identität identifiziert wird. 

Das Ziel der Assoziationsanalyse ist es gute Assoziationsregeln zu finden, basierend auf einer Menge von Transaktionene. Eine generische definition von "interesante Beziehung" ist, das man Gegenstände, die man häufig zusammen in Transaktionen beobachtet. Hier ist ein Beispiel mit zehn Transaktionen.

In [6]:
# we define each transaction as a list
# a set of transactions is a list of lists
data = [['item1', 'item2', 'item3'],
        ['item2', 'item4'],
        ['item1', 'item5'],
        ['item6', 'item7'],
        ['item2', 'item3', 'item4', 'item7'],
        ['item2', 'item3', 'item4', 'item8'],
        ['item2', 'item4', 'item5'],
        ['item2', 'item3', 'item4'],
        ['item4', 'item5'],
        ['item6', 'item7']]
print('Transaktionen (eine pro Zeile):')
print('\n'.join([str(t) for t in data]))

Transaktionen (eine pro Zeile):
['item1', 'item2', 'item3']
['item2', 'item4']
['item1', 'item5']
['item6', 'item7']
['item2', 'item3', 'item4', 'item7']
['item2', 'item3', 'item4', 'item8']
['item2', 'item4', 'item5']
['item2', 'item3', 'item4']
['item4', 'item5']
['item6', 'item7']


In diesem Beispiel tauchen die Gegenstände item2, item3 und item4 oft zusammen auf, was auf eine interesante Beziehung hindeutet. Im folgenden erklären wir wie man effizient solche Beziehungen findet, hieraus Regeln erstellt und die Güte der Regeln bewertet.

## Der Apriori-Algorithmus

Der Apriori-Algorithmus ist ein vom prinzip sehr einfacher, aber denoch mächtiger Algorithmus um Assoziation in Daten zu suchen.

### Support und Frequent Itemsets

Der Apriori-Algorithmus basiert auf dem Konzept des *Support* von Itemsets $IS \subseteq I$. Der Support eines Itemsets $IS$ ist definiert als der Anteil der Transaktionen, in denen alle Gegenstände des Itemsets vorkommen. Es gilt also

$$support(IS) = \frac{|\{t \in T: IS \subseteq t\}|}{|T|}.$$

Der Support ist sehr ähnlich zur generischen Definition von "interesanten Beziehungen" die wir weiter oben benutzt haben, da der Support ein Maß für die Häufigkeit ist, mit der Kombinationen von Gegenständen auftauchen. Entsprechend kann man den Support als indirektes Maß für die Interessantheit sehen. Was noch fehlt ist ein Mindestmaß an Interesantheit, so dass eine Kombination von Gegenständen relevant genug ist um diese als Grundlage für Assoziationsregeln zu nutzen. Es ist nur logisch, dass man hierfür einen Grenzwert für den Support festlegt: ist dieser Grenzwert überschritten, ist ein Itemset interessant und wir sprechen von einem *Frequent Itemset*. Formal nennen wir ein Itemset $IS$  *frequent*, wenn $support(IS) \geq minsupp$ für einen Grenzwert $minsupp \in [0,1]$. 

Im obigen Beispiel hat das Itemset der Gegenstände item2, item3 und item4 eine Support von

$support(\{item2, item3, item4\}) = \frac{3}{10}$,

da alle drei Gegenstände zusammen in drei von zehn Transaktion vorkommen. Wenn wir also $minsupp=0.3$ wählen, wäre das Itemset $\{item2, item3, item4\}$ ein Frequent Itemset. Insgesamt finden wir mit diesem Grenzwert die folgenden Frequent Itemsets.

In [8]:
# we need pandas and mlxtend for the association rules
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.preprocessing import TransactionEncoder
pd.set_option('display.latex.repr', True)

# we first need to create a one-hot encoding of our transactions
te = TransactionEncoder()
te_ary = te.fit_transform(data)
data_df = pd.DataFrame(te_ary, columns=te.columns_)

# we can the use this function to determine all itemsets with at least 0.3 support
# what apriori is will be explained later
frequent_itemsets = apriori(pd.DataFrame(data_df), min_support=0.3, use_colnames=True)
frequent_itemsets

,support,itemsets
0,0.6,(item2)
1,0.4,(item3)
2,0.6,(item4)
3,0.3,(item5)
4,0.3,(item7)
5,0.4,"(item2, item3)"
6,0.5,"(item2, item4)"
7,0.3,"(item3, item4)"
8,0.3,"(item2, item3, item4)"


### Ableiten von Regeln

Ein Frequent Itemset ist noch keine Assoziationsregel, da es weder eine Bedingung $X$ noch eine Folgerung $Y$ aus denen eine Regel $X \Rightarrow Y$ entsteht. Man kann jedoch relativ einfach solche Regeln aus den Frequent Itemsets ableiten. Hierzu betrachtet man alle möglichen Partionierungen eines Frequent Itemsets, also alle Kombinationen $X, Y \subset IS$, so dass $X \cup Y = IS$ (jeder Gegenstand kommt in X oder Y vor) und $X \cap Y = \emptyset$ (kein Gegenstand kommt in X und Y vor). 

Im unserem Beispiel können wir acht Regeln aus dem Itemset $\{item2, item3, item4\}$ ableiten.

In [9]:
from mlxtend.frequent_patterns import association_rules

association_rules(frequent_itemsets.iloc[8:9, :], metric="confidence",
                  min_threshold=0.0, support_only=True)[['antecedents', 'consequents']]

,antecedents,consequents
0,"(item2, item3)",(item4)
1,"(item2, item4)",(item3)
2,"(item3, item4)",(item2)
3,(item2),"(item3, item4)"
4,(item3),"(item2, item4)"
5,(item4),"(item2, item3)"


Es gibt noch zwei weitere Regeln, wenn wir die leere Menge als Bedingung oder Folgerung zulassen: 

- $\emptyset \Rightarrow \{item2, item3, item4\}$
- $\{item2, item3, item4\} \Rightarrow \emptyset$

Diese Regeln erlauben aber keine bedeutenden Einblicke, so dass wir sie ignorieren können. 

### Confidence, Lift, und Leverage

Eine noch ungeklärte Frage ist wie wir erkennen welche der Regeln gut sind, oder ob alle Regeln die man aus einem Frequent Itemset ableitet gleich gut sind. Am Beispiel der leeren Menge sieht man recht schnell, dass vermutlich nicht alle Regeln gleich gut sind. Was wir brauchen sind Gütemaße, die es uns erlaubten zwischen bedeutungsvollen Assoziatonen und zufälligen Effekten zu unterscheiden. Hierfür ist der Support nicht ausreichend. Stellen Sie sich zum Beispiel vor, das es etwas Gratis gibt. Diese Gratisproduke werden häufig einfach "mitgenommen" und landen entsprechend oft im Warenkorb und später dann in unseren Transaktionen. Diese Gratisprodukte haben einen hohen Support und sind mit hoher Wahrscheinlichkeit ein Teil vieler Frequent Itemsets. Die Gratisprodukte sind aber nicht mit dem Rest von Einkauf assoziiert, die Beziehung ist also zufällig. Eine weitere schwäche des Supports ist das dieser nicht von der Regel abhängt, insbesondere ist der Support einer Regel $X \Rightarrow Y$ immer gleich der Regel $Y \Rightarrow X$ in der einfach Bedingung und Folgerung vertauscht sind. Da aber kausale zusammenhänge häufig eine Richtung haben, macht diese symmetrie in vielen Fällen keine Sinn. Es macht zum Beispiel durchaus Sinn, einem Kunden der eine Spielekonsole in den Warenkorb legt, ein Spiel für diese Konsole vorzuschlagen. Wenn jemand aber ein Spiel in den Warenkorb legt, ist die Konsole im normalfall bereits vorhanden, und es würde keinen Sinn machen diese vorzuschlagen. 

Die Maße *Confidence*, *Lift*, und *Leverage* helfen dabei zu erkennen ob Regeln zufällig sind oder ob es sich um sinnvolle Assoziationen handelt. Die Confidence ist definiert als

$$confidence(X \Rightarrow Y) = \frac{support(X \cup Y)}{support(X)}.$$

Es handelt sich bei der Confidence also um das Verhältnis die Bedindung $X$ und Folgerung $Y$ zusammen in einer Transaktion zu der Beobachtung von nur der Bedingung $X$. Eine hohe Confidenz deutet drauf hin, dass Bedindung selten ohne die Folgerung zu beobachten ist, was auf einen starken Zusammenhang hindeutet.

Der Lift ist definiert als

$$lift(X \Rightarrow Y) = \frac{support(X \cup Y)}{support(X) \cdot support(Y)}.$$

Der Lift misst das Verhältnis der Erwartung das die Bedingung $X$ und die Folgerung $Y$ zusammen zu beobachten im Verhältnis zur Erwartung das $X$ und $Y$ unabhängig voneinander auftauchen. Der Support kann nämlich auch einfach als Schätzer für die den Erwartungswert ein Itemset zu beobachten eines Itemsets interpretiert werden. Daher ist $support(X \cup Y)$ ist nichts anderes als der Erwartungswert von $X$ und $Y$ zusammen und $support(X) \cdot support(Y)$ der Erwartungswert unter der Annahme das $X$ und $Y$ unabhängig voneinander sind. Unabhängigkeit ist das direkte gegenteil von assoziiert. Es folgt daraus das der Lift misst, wie viel wahrscheinlicher es ist, das die Teile einer Regel $X$ und $Y$ in irgendeiner weise von einander abhängen, als das diese unabhängig sind. Ein Lift von 2 bedeutet also, dass es doppelt so wahrscheinlich ist, dass es eine assoziation gibt, wie das es keine gibt.

Der Leverage ist definiert als

$$leverage(X \Rightarrow Y) = support(X \cup Y) - support(X) \cdot support(Y).$$

Diese Definition ist ähnlich zum Lift, nur dass die Differenz statt dem Verhältnis betrachtet wird. Entsprechend sind sich Lift und Leverage sehr ähnlich. Der unterschied ist, das Leverage tendenziell höhere Werte für Itemsets mit einem hohen Support liefert. 

Um Confidence, Lift und Leverage besser zu verstehen, betrachten wir diese nun am Beispiel der Regeln, die wir aus dem Itemset $\{item2, item3, item4\}$ ableiten können.

In [10]:
# all rules for our data
# we drop the column conviction, because this metric is not covered here
# we also drop all rules from other itemsets than above. 
association_rules(frequent_itemsets, metric="confidence",
                  min_threshold=0.0).drop('conviction', axis=1)[6:].reset_index(drop=True)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage
0,"(item2, item3)",(item4),0.4,0.6,0.3,0.75,1.250000,0.06
1,"(item2, item4)",(item3),0.5,0.4,0.3,0.60,1.500000,0.10
2,"(item3, item4)",(item2),0.3,0.6,0.3,1.00,1.666667,0.12
3,(item2),"(item3, item4)",0.6,0.3,0.3,0.50,1.666667,0.12
4,(item3),"(item2, item4)",0.4,0.5,0.3,0.75,1.500000,0.10
5,(item4),"(item2, item3)",0.6,0.4,0.3,0.50,1.250000,0.06


Basierend auf den Metriken schneidet die Regel $\{item3, item4\} \Rightarrow \{item2\}$ klar am besten ab. Diese Regel hat eine perfekte Confidence fon 1, was nichts anderes bedeutet das item2 in jeder Transaktion vorkommt, in denen item3 und item 4 zusammen vorkommen. Der Lift von 1,66 bedeutet, dass dieses gemeinsame Auftreten 1,66-mal Wahrscheinlicher ist, als es sich durch eine zufällige Beziehung erklären liese. Hier sieht man auch den Unterschied zur Regel  $\{item2\} \Rightarrow \{item3, item4\}$, also die Regel, die wir bekommen wenn wir Bedingung und Folgerung vertauschen. Diese Regel hat zwar den gleichen Lift (und Leverage), die Confidence ist jedoch nur 0,5. Das bedeutet das item2 genauso häufig alleine vorkommt, wie mit item3 und item4 zusammen. Während also die Regel $\{item3, item4\} \Rightarrow \{item2\}$ vermutlich fast jedes mal richtig ist, ist die Regel $\{item2\} \Rightarrow \{item3, item4\}$ in ca. der hälfte der Fälle falsch. 

Das Bespiel zeigt auch die grundlegenden Eigenschaften der drei Maße: Während Lift und Leverage sind symmetrisch, genauso wie der Support. Die Confidence ist das einzige Gütemaße, welches berücksichtigt welcher Teil einer Regel die Bedingung und welcher Teil die Folgerung ist, also das einzige Maß welches die Kausalität berücksichtigt. Daher sollte man die Confidence immer bei der Bewertung von Regeln benutzen. Da Lift und Leverage sich sehr ähnlich verhalten, reicht es in der Regel nur eines der beiden maße zu verwenden. Legt man mehr Wert auf Regeln, die aus Itemsets mit hohem Support gewonnen werden ist Leverage besser, ansonsten sollte man Lift wählen. Der Grund hierfür ist, das es für Lift eine direkt Interpretation gibt, also wie viel wahrscheinlicher etwas ist als es sich durch den reinen Zufall erklären lässt. 

### Exponential Growth

We have now shown how we can determine rules: we need to find frequent itemsets and can then derive rules from these sets. Unfortunately, finding the frequent itemsets is not trivial. The problem is that the number of itemsets grows exponentially with the number of items. The possible itemsets are the powerset $\mathcal{P}$ of $I$, which means there are $|\mathcal{P}(I)|=2^{|I|}$ possible itemsets. Obviously, there are only very few use cases, where we would really need to consider all items, because often shorter rules are preferable. Unfortunately, the growth is still exponential, even if we restrict the size. We can use the binomial coefficient

$${|I| \choose k} = \frac{|I|!}{(|I|-k)!k!}$$


to calculate the number of itemsets of size k. For example, if we have |I|=100 items, there are already 161,700 possible itemsets of size $k=3$. We can generate eight rules for each of these itemsets, thus we already have 1,293,600 possible rules. If we ignore rules with the empty itemset, we still have 970,200 possible rules. 

Thus, we need a way to search the possible itemsets strategically to deal with the exponential nature, as attempt to find association rules could easily run out of memory or require massive amounts of computational resources, otherwise. 

### The Apriori Property

Finally, we come to the reason why this approach is called the Apriori algorithm. 

> **Apriori Property**
>
> Let $IS \subseteq I$ be a frequent itemset. All subsets $IS' \subseteq IS$ are also frequent and $support(IS') \geq support(IS)$.

This property allows us to search for frequent itemsets in a bounded way. Instead of calculating all itemsets and then checking if they are frequent, we can *grow* frequent itemsets. Since we know that all subsets of a frequent itemset must be frequent, we know that any itemset that contains a non-frequent subset cannot be frequent. We use this to prune the search space as follows.

1. Start with itemsets of size $k=1$.
2. Drop all itemsets that do not have the minimal support, so that we only have frequent itemsets left.
3. Create all combinations of the currently known frequent itemsets of size $k+1$
4. Repeat steps 2 and 3 until
  - No frequent itemsets of length $k+1$ are found
  - A threshold for $k$ is reached, i.e., a maximal length of itemsets.
  
This algorithm can still be exponential. For example, if all transactions contain all items, all possible itemsets are frequent and we still have exponential growth. However, in practice this algorithm scales well, if the support is sufficiently high. 

For example, let us consider how we can grow frequent itemsets with $minsupp=0.3$ for our example data. Here is the data again. 